In [1]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
datasets=pd.read_csv("Social_Network_Ads.csv")

In [3]:
datasets.head(133)

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
128,15722758,Male,30,17000,0
129,15792102,Female,26,84000,0
130,15675185,Male,31,58000,0
131,15801247,Male,33,31000,0


In [4]:
datasets=pd.get_dummies(datasets,drop_first=True)

In [5]:
datasets

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=datasets[['Age', 'EstimatedSalary','Gender_Male']]
dep=datasets['Purchased']

In [7]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 





grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 


C:\Users\rajah\anaconda3\envs\pugazh\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  34 out of  36 | elapsed:    5.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.3s finished
C:\Users\rajah\anaconda3\envs\pugazh\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [13]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)



C:\Users\rajah\anaconda3\envs\pugazh\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [14]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}: 0.9171245421245421


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[73  6]
 [ 4 37]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

C:\Users\rajah\anaconda3\envs\pugazh\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


0.9658845322630442

In [18]:
print(grid.best_params_)

{'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}


In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.043950,0.007611,0.016723,7.040999e-03,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.851064,0.882677,0.879431,0.870986,0.014224,8
1,0.361504,0.029679,0.023629,1.218063e-02,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.851064,0.872288,0.913392,0.878815,0.025880,6
2,0.047910,0.006011,0.015938,6.877413e-03,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.838689,0.861910,0.876890,0.859090,0.015735,10
3,0.311613,0.016634,0.026466,1.207043e-02,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.851064,0.872288,0.924486,0.882500,0.030853,3
4,0.044052,0.006229,0.007032,1.486660e-04,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.871453,0.861345,0.879431,0.870745,0.007387,9
5,0.197904,0.010084,0.015620,6.743496e-07,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.873082,0.872288,0.935484,0.893545,0.029578,1
6,0.021061,0.008930,0.001995,2.821029e-03,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.851064,0.872288,0.891740,0.871624,0.016627,7
7,0.181469,0.006880,0.015749,4.020669e-03,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.873082,0.872288,0.902192,0.882487,0.013900,4
8,0.026035,0.007364,0.005207,7.363897e-03,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.827213,0.861345,0.866462,0.851586,0.017451,12
9,0.179570,0.012754,0.010414,7.364010e-03,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.873082,0.872288,0.902192,0.882487,0.013900,4


In [21]:
table_pr=pd.DataFrame()

In [22]:
print(X_test)

[[-0.77101313  0.49720103  0.99288247]
 [ 0.0133054  -0.57280368 -1.00716855]
 [-0.28081405  0.15017248  0.99288247]
 [-0.77101313  0.26584866 -1.00716855]
 [-0.28081405 -0.57280368 -1.00716855]
 [-1.06513258 -1.44037507  0.99288247]
 [-0.67297331 -1.5849703  -1.00716855]
 [-0.18277423  2.14558666  0.99288247]
 [-1.94749093 -0.05226085 -1.00716855]
 [ 0.89566375 -0.775237    0.99288247]
 [-0.77101313 -0.60172273  0.99288247]
 [-0.96709276 -0.42820845 -1.00716855]
 [-0.08473441 -0.42820845  0.99288247]
 [ 0.11134522  0.20801057  0.99288247]
 [-1.7514113   0.46828198  0.99288247]
 [-0.5749335   1.36477242 -1.00716855]
 [-0.08473441  0.20801057  0.99288247]
 [-1.84945111  0.43936294  0.99288247]
 [ 1.67998229  1.74072002 -1.00716855]
 [-0.28081405 -1.38253697  0.99288247]
 [-0.28081405 -0.65956082 -1.00716855]
 [ 0.89566375  2.14558666 -1.00716855]
 [ 0.30742485 -0.54388463 -1.00716855]
 [ 0.89566375  1.01774386  0.99288247]
 [-1.45729185 -1.2090227  -1.00716855]
 [ 1.09174339  2.05882953